<h1 style="text-align: center; font-size: 60px;"> Интеллектуальный анализ и обработка данных</h1>
<hr>
<h2 style="text-align: center;font-size: 40px;"> ЛАБОРАТОРНАЯ РАБОТА №2 <br>
 Предиктивный анализ данных на основе регрессионных моделей SVM, CART и ансамблевых моделей</h2>
 <hr>

<h3 style="text-align: center; font-size: 30px;">Исполнители</h3>
<p><b>Корзун Николай Вадимович</b> <a href="https://github.com/nikoloskorzun">GitHub</a></p>


<hr>
<h3 style="text-align: center; font-size: 30px;">Постановка задачи</h3>

<p>Дан многомерный размеченный набор данных, для которого был выполнен разведочный и предиктивный анализ в ЛР1. Необходимо выполнить предиктивный анализ данных на основе регрессионных моделей по вариантам. Анализ выполняется для четырех наборов данных из ЛР1: для исходного набора данных, преобразованного набора исходного данных, построенного набора данных и
преобразованного построенного набора данных.</p> 
<p>Предиктивный анализ выполняется с помощью неансамблевой и ансамблевой модели по варианту. Результаты сравниваются и выбирается лучший вариант по  модели и анализируемому набору данных.</p>
<p>Для выбранной комбинации набора данных и предиктивной модели
выполняется поиск гиперпараметра на обучающем и тестовом наборах.</p>
<p>Полученный результат на тестовом наборе сравнивается с результатом наилучшей предиктивной модели/набора, найденными в ЛР1.</p>
<p>Сделать выводы по проведенному анализу.</p>

<p><b>Вариант №12</b></p>
<p>Набор данных различных форм зданий, различающихся по площади остекления, распределению площади остекления, ориентации и другим параметрам. <br>В зависимости от параметров здания других признаков аргументов. Построить регрессионную модель оценки целевого признака требований к тепловой нагрузке зданий:</p>

<ol>
    <li>X1 вещественный, относительная компактность</li>
    <li>X2 вещественный, площадь поверхности</li>
    <li>X3 вещественный, площадь сплошной стены</li>
    <li>X4 вещественный, зона крыши</li>
    <li>X5 вещественный, общая высота</li>
    <li>X6 целочисленный, ориентация</li>
    <li>X7 вещественный, площадь сплошного остекления</li>
    <li>X8 целочисленный, распределение площади остекления</li>    
    <li>Y1 целевой признак, вещественный, отопительная нагрузка
    <ol>
            <li>Модель CART</li>
            <li>Градиентный бустинг</li>
    </ol>
    </li>
</ol>



In [1]:
names = ['относительная компактность', 'площадь поверхности', 'площадь сплошной стены',
         'зона крыши', 'общая высота', 'ориентация', 'площадь сплошного остекления',
         'распределение площади остекления', 'отопительная нагрузка']
target_feature = names[-1]
datasets = []
models = []

<h3 style="text-align: center; font-size: 30px;">Выполнение задачи</h3>
<h4 style="text-align: center; font-size: 24px;">Этап 2<br>Загрузка датасета</h4>

In [2]:
import numpy as np

import pandas as pd




from sklearn import metrics

from dataset import Dataset
from model import Model
from utils import print_results


In [3]:
df = pd.read_excel("../../V12.xlsx", names = names)

In [4]:
datasets = []
datasets.append(Dataset(df, [target_feature]))
datasets.append(Dataset(df, [target_feature]))
datasets.append(Dataset(df, [target_feature]))
datasets.append(Dataset(df, [target_feature]))
datasets[0].name = "без изменений"
datasets[1].name = "без изменений, нормализован"
datasets[2].name = "очищенный"
datasets[3].name = "очищенный, нормализован"


In [5]:
from sklearn.preprocessing import Normalizer

def slice_1_lab(df: pd.DataFrame):
    dfc= df.copy()
    if "площадь сплошной стены" in df.columns and "площадь поверхности" in  df.columns:
        dfc["отношение стенной площади ко всей площади дома"] = df["площадь сплошной стены"] / df["площадь поверхности"]
    if "площадь сплошного остекления" in df.columns and "площадь поверхности" in  df.columns:
        dfc["отношение остекленния ко всей площади дома"] = df["площадь сплошного остекления"] / df["площадь поверхности"]
    if "зона крыши" in df.columns and "площадь поверхности" in  df.columns:
        dfc["отношение площади крыши ко всей площади дома"] = df["зона крыши"] / df["площадь поверхности"]

    lab1_names = ['относительная компактность', 'площадь поверхности',
                  'отопительная нагрузка', 'площадь сплошного остекления',
                  'отношение стенной площади ко всей площади дома', 'зона крыши',
                  'отношение остекленния ко всей площади дома', 'общая высота',
                  'отношение площади крыши ко всей площади дома']
    
    return dfc[lab1_names]
def scale(df: pd.DataFrame):
    df_c = pd.DataFrame(Normalizer().fit_transform(df))
    df_c.columns = df.columns
    return df_c



In [6]:
datasets[2].performe_operation(slice_1_lab)
datasets[3].performe_operation(slice_1_lab)
datasets[1].performe_operation(scale)
datasets[3].performe_operation(scale)

In [7]:
for ds in datasets:
    print(ds)

Датасет без изменений c 8 + 1 признаками
Датасет без изменений, нормализован c 8 + 1 признаками
Датасет очищенный c 8 + 1 признаками
Датасет очищенный, нормализован c 8 + 1 признаками


In [8]:
def r2_score(y_predict, y_real):
        return metrics.r2_score(y_predict, y_real)
def rmse(y_predict, y_real):
        return np.sqrt(metrics.mean_squared_error(y_predict, y_real))
scores = {"R²":r2_score, "RMSE":rmse}

Дерево решений (CART)

In [9]:
from sklearn import tree
models = []
models.append(Model(tree.DecisionTreeRegressor()))
models[0].name = "дерево решений" 



In [10]:
from sklearn.ensemble import GradientBoostingRegressor
models.append(Model(GradientBoostingRegressor(random_state=1337)))
models[1].name = "градиентный бустинг" 

In [11]:
results = []
for dataset in datasets:
    #print(dataset)

    for model in models:
        model.fit(dataset.X_train, dataset.Y_train)
        #print(model)
        for test in ["train", "test", "valid"]:
            temp = model.test_model(dataset, scores, test_type=test)
            results.append([temp, dataset.name, model.name, test])

Датасет без изменений c 8 + 1 признаками
Модель дерево решений
Модель градиентный бустинг
Датасет без изменений, нормализован c 8 + 1 признаками
Модель дерево решений
Модель градиентный бустинг
Датасет очищенный c 8 + 1 признаками
Модель дерево решений


S:\anaconda\envs\study\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
S:\anaconda\envs\study\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
S:\anaconda\envs\study\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


Модель градиентный бустинг
Датасет очищенный, нормализован c 8 + 1 признаками
Модель дерево решений
Модель градиентный бустинг


S:\anaconda\envs\study\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [12]:
results_r2 = sorted(results, key=lambda x: x[0]['R²'], reverse=True)
results_rmse = sorted(results, key=lambda x: x[0]['RMSE'], reverse=True)
print_results(results_r2)
print("\n\n")
print_results(results_rmse)


Датасет без изменений, нормализован метод дерево решений на выборке test дал результаты {'R²': 0.996351299177121, 'RMSE': np.float64(0.6619863850759876)}
Датасет без изменений метод дерево решений на выборке test дал результаты {'R²': 0.99635822633648, 'RMSE': np.float64(0.6616537722145276)}
Датасет очищенный метод дерево решений на выборке test дал результаты {'R²': 0.996358332441791, 'RMSE': np.float64(0.6616616234119246)}
Датасет очищенный, нормализован метод дерево решений на выборке test дал результаты {'R²': 0.996358332441791, 'RMSE': np.float64(0.6616616234119246)}
Датасет очищенный метод дерево решений на выборке valid дал результаты {'R²': 0.9974812833203024, 'RMSE': np.float64(0.5449276849437753)}
Датасет очищенный, нормализован метод дерево решений на выборке valid дал результаты {'R²': 0.9974812833203024, 'RMSE': np.float64(0.5449276849437753)}
Датасет без изменений метод дерево решений на выборке valid дал результаты {'R²': 0.9974826052004907, 'RMSE': np.float64(0.54478467